In [1]:
# Python standard library packages to do file/folder manipulations,
# pickle is a package to store python variables
import glob, os, pickle, sys

# time grabs current clock time and copy to safely make copies of large 
# variables in memory.
import time, copy 

# datetime package is used to get and manipulate date and time data
from datetime import datetime

# this packages helps load and save .mat files older than v7
import hdf5storage 

# numpy works with arrays, pandas used to work with fancy numpy arrays
import numpy as np
import pandas as pd

# matplotlib is used to plot and animate to make movies
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# moviepy helps open the video files in Python
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.video.io.bindings import mplfig_to_npimage

# Scikit-learn is a go-to library in Python for all things machine learning
from sklearn.decomposition import PCA

# tqdm helps create progress bars in for loops 
from tqdm import tqdm 

# Scipy is a go-to scientific computing library. We'll use it for median filtering. 
from scipy.ndimage import median_filter

# Configuring matplotlib to show animations in a colab notebook as javascript 
# objects for easier viewing. 
from matplotlib import rc
rc('animation', html='jshtml')

# Creating an mmpy project directory

Now that we have two low dimensional time series which **may** not set Google servers on fire, we will create our project directory for running the `motionmapperpy` pipeline on the data we have. Having a project directory is awesome, as it helps us stay organized when working with big datasets and multiple files. It allows datasets to be easily referenced and loaded without exhausting memory, and we can store pipeline outputs in well-defined and easy to read files.


Lets start by importing `motionmapperpy` and creating a project directory. We've already created this, so no need to do anything beside set path.

In [2]:
import motionmapperpy as mmpy


projectPath = 'mmpy_lts_all'

# This creates a project directory structure which will be used to store all motionmappery pipeline
# related data in one place.

# mmpy.createProjectDirectory(projectPath)


Now lets store the two low-d time series in **projs_list** to the *`Projections`* folder in the project directory.



In [ ]:
# for i,projs in enumerate(projs_list):
#     hdf5storage.savemat('%s/Projections/%s_pcaModes.mat'%(projectPath, datasetnames[i]), {'projections':projs})

We'll now go through `mmpy` parameters. They are a handful and can be overwhelming, but they are very easy to understand! 

Parameters are cruicial to `mmpy` as they lay out some hard-coded choices we need to make when running this pipeline. I will explain each parameter as we encounter them in the cell below, so please read through this cell below as you run it.


In [3]:
"""2. Setup run parameters for MotionMapper."""

#% Load the default parameters.
parameters = mmpy.setRunParameters() 


In [4]:
parameters

{'numProcessors': 16,
 'numPeriods': 25,
 'omega0': 5,
 'samplingFreq': 100,
 'minF': 1,
 'maxF': 50,
 'tSNE_method': 'barnes_hut',
 'perplexity': 32,
 'embedding_batchSize': 128000,
 'maxOptimIter': 500,
 'trainingSetSize': 64000,
 'maxNeighbors': 200,
 'kdNeighbors': 5,
 'training_perplexity': 20,
 'training_numPoints': 32000,
 'minTemplateLength': 1,
 'waveletDecomp': True,
 'useGPU': -1,
 'n_neighbors': 15,
 'train_negative_sample_rate': 5,
 'embed_negative_sample_rate': 1,
 'min_dist': 0.1,
 'umap_output_dims': 2,
 'n_training_epochs': 100,
 'rescale_max': 100,
 'method': 'TSNE',
 'projectPath': 'mmpy_lts_1d'}

In [9]:

# %%%%%%% PARAMETERS TO CHANGE %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# These need to be revised everytime you are working with a new dataset. #
import glob
import h5py
import natsort
parameters.projectPath = 'mmpy_lts_1d'#parameters.projectPath #% Full path to the project directory.


parameters.method = 'TSNE' #% We can choose between 'TSNE' or 'UMAP'

parameters.minF = 1        #% Minimum frequency for Morlet Wavelet Transform

parameters.maxF = 50       #% Maximum frequency for Morlet Wavelet Transform,
                           #% usually equal to the Nyquist frequency for your
                           #% measurements.

parameters.samplingFreq = 100    #% Sampling frequency (or FPS) of data.

parameters.numPeriods = 25       #% No. of dyadically spaced frequencies to
                                 #% calculate between minF and maxF.
projectionFiles = glob.glob(parameters.projectPath + "/Projections/*pcaModes.mat")
print("Found {} projection files".format(len(projectionFiles)))
print(f"Project path: {parameters.projectPath}")
projectionFiles = natsort.natsorted(projectionFiles)
with h5py.File(projectionFiles[0], "r") as f:
    m = f["projections"][:].T
# parameters.pcaModes = #comps_above_thresh #% Number of low-d features.
parameters.pcaModes = m.shape[1]  #%Number of PCA projections in saved files.
parameters.numProjections = parameters.pcaModes
parameters.numProcessors = -1     #% No. of processor to use when parallel
                                 #% processing for wavelet calculation (if not using GPU)  
                                 #% and for re-embedding. -1 to use all cores 
                                 #% available.

parameters.useGPU = 0           #% GPU to use for wavelet calculation, 
                                 #% set to -1 if GPU not present.

parameters.training_numPoints = 3000    #% Number of points in mini-trainings.


# %%%%% NO NEED TO CHANGE THESE UNLESS MEMORY ERRORS OCCUR %%%%%%%%%%

parameters.trainingSetSize = 5000  #% Total number of training set points to find. 
                                 #% Increase or decrease based on
                                 #% available RAM. For reference, 36k is a 
                                 #% good number with 64GB RAM.

parameters.embedding_batchSize = 30000  #% Lower this if you get a memory error when 
                                        #% re-embedding points on a learned map.

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Found 48 projection files
Project path: mmpy_lts_1d


Above covers usually relevant parameters when using `mmpy`. However, there are parameters associated with tSNE and UMAP implementations, such as below, which aren't usually required to be changed.

In [10]:
# %%%%%%% tSNE parameters %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#% can be 'barnes_hut' or 'exact'. We'll use barnes_hut for this tutorial for speed.
parameters.tSNE_method = 'barnes_hut' 

# %2^H (H is the transition entropy)
parameters.perplexity = 32

# %number of neigbors to use when re-embedding
parameters.maxNeighbors = 200

# %local neighborhood definition in training set creation
parameters.kdNeighbors = 5

# %t-SNE training set perplexity
parameters.training_perplexity = 20


# %%%%%%%% UMAP Parameters %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Size of local neighborhood for UMAP.
n_neighbors = 15

# Negative sample rate while training.
train_negative_sample_rate = 5

# Negative sample rate while embedding new data.
embed_negative_sample_rate = 1

# Minimum distance between neighbors.
min_dist = 0.1


## 3.1&nbsp; Visualizing wavelet amplitudes

This section is not required to be run by motionmapperpy, but we'll go through it to visualize spectrograms on one of the low-dimensional time series.

We'll use `mmpy.findWavelets` function to obtain the waveletes, and plot the obtained spectrogram for each feature/projection. 

In [11]:
import pathlib
import h5py
import natsort
# wlets, freqs = mmpy.findWavelets(projs_list[0], projs_list[0].shape[1], parameters.omega0, parameters.numPeriods, parameters.samplingFreq, parameters.maxF, parameters.minF, parameters.numProcessors, parameters.useGPU)
projectionFiles = glob.glob(parameters.projectPath + "/Projections/*pcaModes.mat")
projectionFiles = natsort.natsorted(projectionFiles)
projectionFile = projectionFiles[0]
wavelet_path = f"{parameters.projectPath}/Wavelets/{pathlib.Path(projectionFile).stem}-wavelets.mat"
with h5py.File(wavelet_path, "r") as hfile:
    wlets = hfile["wavelets"][:].T


In [ ]:
with open('frequencies.pickle', 'rb') as handle:
    freqs = pickle.load(handle)

In [ ]:
# wlets[:600,25*i:25*(i+1)].T.shape
# plt.imshow(wlets[:600,25*i:25*(i+1)].T, aspect='auto', cmap='jet')
# del fig, ax


In [ ]:
fig, axes = plt.subplots(24, 1)

for i, ax in enumerate(axes.flatten()):
  # print(f'shape: {wlets[:1000,25*i:25*(i+1)].shape}')
  ax.imshow(wlets[25*i:25*(i+1),:(10*100)], cmap='PuRd')
  ax.set_yticks([0, 5, 10, 15, 20, 24])
  ax.set_yticklabels(['%0.1f'%freqs[j] for j in [0, 5, 10, 15, 20, 24]])
  if i == 11:
    pass
    # ax.set_ylabel("Frequencies (hz)", fontsize=14)
  if i != 23:
    ax.get_yaxis().set_visible(False)
    ax.set_xticklabels([])
  # ax.set_title('Projection #%i'%(i+1))
ax.set_xlabel('Frames', fontsize=14)
plt.tight_layout()
plt.savefig("tmp2.png",dpi=300)
plt.close()

As we can see, our low-d time series is soon dwarfed by the 25-dimensional wavelet amplitudes obtained for each low-d feature! This is why its wise to spend some time reducing the dimensionality of our original data, as much as we can. 


# 5.&nbsp; Creating a training set and embedding it using tSNE/UMAP 

Even though we are working with toy datasets, we have two extremely high dimensional timeseries we're using to create a smaller and more interpretable representation. tSNE and UMAP both need to compute all-to-all distances in high-dimensional space to find neighboring points and embed them closely on this low-dimensional space we're building. This computation can quickly exhaust memory (RAM) and scale exponentially with datapoints. 

To navigate this challenge, we do a subsampling procedure to create a training set, and use tSNE or UMAP to create training embeddings. All of this is done in the cell below.

**Time taken** : TSNE 86 sec | UMAP 44 sec


In [ ]:
import glob
import natsort
import h5py
projectionFiles = glob.glob(parameters.projectPath + "/Projections/**pcaModes.mat")
projectionFiles = natsort.natsorted(projectionFiles)
print(projectionFiles[0])
with h5py.File(projectionFiles[0], "r") as f:
    m = f["projections"][:].T

# %%%%%
print(m.shape)
parameters.pcaModes = m.shape[1]  #%Number of PCA projections in saved files.
parameters.numProjections = parameters.pcaModes
# %%%%%
del m

print(datetime.now().strftime("%m-%d-%Y_%H-%M"))
print("tsneStarted")

if parameters.method == "TSNE":
    if parameters.waveletDecomp:
        tsnefolder = parameters.projectPath + "/TSNE/"
    else:
        tsnefolder = parameters.projectPath + "/TSNE_Projections/"
elif parameters.method == "UMAP":
    tsnefolder = parameters.projectPath + "/UMAP/"
for i in range(len(projectionFiles)):
    if not os.path.exists(tsnefolder + "training_tsne_embedding.mat"):
        print("Calculating wavelets...")
        mmpy.get_wavelets(projectionFiles, parameters, i)
        print(datetime.now().strftime("%m-%d-%Y_%H-%M"))

In [ ]:
# t1 = time.time()

# mmpy.subsampled_tsne_from_projections(parameters, parameters.projectPath)

# print('Done in %i seconds.'%(time.time()-t1))


Note that the `training set` and `training embedding` are both save in the `project_directory/TSNE` or `project_directory/UMAP` directories depending on which method you're using. We'll load the training embedding below and plot it. You can play around with the sigma value here to change the coarseness of the density map. 

In [ ]:
trainy = hdf5storage.loadmat('%s/%s/training_embedding.mat'%(parameters.projectPath, parameters.method))['trainingEmbedding']
m = np.abs(trainy).max()


sigma=2.0
_, xx, density = mmpy.findPointDensity(trainy, sigma, 511, [-m-20, m+20])


fig, axes = plt.subplots(1, 2, figsize=(12,6))
axes[0].scatter(trainy[:,0], trainy[:,1], marker='.', c=np.arange(trainy.shape[0]), s=1)
axes[0].set_xlim([-m-20, m+20])
axes[0].set_ylim([-m-20, m+20])

axes[1].imshow(density, cmap=mmpy.gencmap(), extent=(xx[0], xx[-1], xx[0], xx[-1]), origin='lower')
plt.savefig("tmp3.png",dpi=300)

On the left, we see a scatter plot and on the right, we see a Gaussian kernel convolved density estimation of these points. Does it surprise you? What does changing the sigma value do?  


# 6.&nbsp; Finding embeddings for all data

Now, we can find embeddings for our entire dataset! We'll use the `mmpy.findEmbeddings` function which requires the training set and the 2-d embeddings we find in the last step, and the high-d 'projections' time series for each dataset. We'll save the obtained embeddings for each dataset neatly in the Projections folder so that we can reference them later.

**Running time** : TSNE 19 mins | UMAP 3 mins

In [ ]:
# #tsne takes 19 mins
# tall = time.time()

# import h5py
# tfolder = parameters.projectPath+'/%s/'%parameters.method

# # Loading training data
# with h5py.File(tfolder + 'training_data.mat', 'r') as hfile:
#     trainingSetData = hfile['trainingSetData'][:].T

# # Loading training embedding
# with h5py.File(tfolder+ 'training_embedding.mat', 'r') as hfile:
#     trainingEmbedding= hfile['trainingEmbedding'][:].T

# if parameters.method == 'TSNE':
#     zValstr = 'zVals' 
# else:
#     zValstr = 'uVals'

# projectionFiles = glob.glob(parameters.projectPath+'/Projections/*pcaModes.mat')
# for i in range(len(projectionFiles)):
#     print('Finding Embeddings')
#     t1 = time.time()
#     print('%i/%i : %s'%(i+1,len(projectionFiles), projectionFiles[i]))


#     # Skip if embeddings already found.
#     if os.path.exists(projectionFiles[i][:-4] +'_%s.mat'%(zValstr)):
#         print('Already done. Skipping.\n')
#         continue

#     # load projections for a dataset
#     projections = hdf5storage.loadmat(projectionFiles[i])['projections']

#     # Find Embeddings
#     zValues, outputStatistics = mmpy.findEmbeddings(projections,trainingSetData,trainingEmbedding,parameters,projectionFiles[i])

#     # Save embeddings
#     hdf5storage.write(data = {'zValues':zValues}, path = '/', truncate_existing = True,
#                     filename = projectionFiles[i][:-4]+'_%s.mat'%(zValstr), store_python_metadata = False,
#                       matlab_compatible = True)
    
#     # Save output statistics
#     with open(projectionFiles[i][:-4] + '_%s_outputStatistics.pkl'%(zValstr), 'wb') as hfile:
#         pickle.dump(outputStatistics, hfile)

#     del zValues,projections,outputStatistics

# print('All Embeddings Saved in %i seconds!'%(time.time()-tall))


In [ ]:
files = glob.glob(parameters.projectPath+'/Projections/*_%s.mat'%("zVals"))
ally = hdf5storage.loadmat(files[0])['zValues']

We can visualize the obtained embeddings by calling the cell below.

In [ ]:
# load all the embeddings
# for i in glob.glob(parameters.projectPath+'/Projections/*zVals.mat'):
  # ally = hdf5storage.loadmat(i)['zValues']

m = np.abs(ally).max()

sigma=2.0
_, xx, density = mmpy.findPointDensity(ally, sigma, 511, [-m-10, m+10])


fig, axes = plt.subplots(1, 2, figsize=(12,6))
axes[0].scatter(ally[:,0], ally[:,1], marker='.', c=np.arange(ally.shape[0]), s=1)
axes[0].set_xlim([-m-20, m+20])
axes[0].set_ylim([-m-20, m+20])

axes[1].imshow(density, cmap=mmpy.gencmap(), extent=(xx[0], xx[-1], xx[0], xx[-1]), origin='lower')
plt.savefig("tmp4.png",dpi=300)

# 7.&nbsp; Watershed transform on the density map. 

There is another handy function in `motionmapperpy` called `findWatershedRegions`. This will do an iterative watershed transform on the behavioral density map until the given `minimum_regions` are found in the density map.

It saves watershed transformed output of the embedding in `project_director/UMAP/zVals_wShed_groups.mat` file.


In [ ]:
# import matplotlib.pyplot as plt
# startsigma = 4.2 if parameters.method == 'TSNE' else 1.0
# mmpy.findWatershedRegions(parameters, minimum_regions=10, startsigma=startsigma, pThreshold=[0.33, 0.67],
#                      saveplot=True, endident = '*_pcaModes.mat')
mmpy.findWatershedRegions(
    parameters,
    minimum_regions=20,
    startsigma=1,
    pThreshold=[0.33, .67],
    saveplot=True,
    endident="*-pcaModes.mat",
)
# from IPython.display import Image
# Image(glob.glob('%s/%s/zWshed*.png'%(parameters.projectPath, parameters.method))[0])

# 8.&nbsp; Ethograms and videos

We can now create ethograms using the watershed region time series created in the last step. 

In [ ]:
wshedfile = hdf5storage.loadmat('%s/%s/zVals_wShed_groups.mat'%(parameters.projectPath, parameters.method))

wregs = wshedfile['watershedRegions'].flatten()
ethogram = np.zeros((wregs.max()+1, len(wregs)))

for wreg in range(1, wregs.max()+1):
  ethogram[wreg, np.where(wregs==wreg)[0]] = 1.0

In [ ]:

ethogram_2 = np.split(ethogram.T, np.cumsum(wshedfile['zValLens'][0].flatten())[:-1])

In [ ]:
[ethogram.shape for ethogram in ethogram]
wshedfile['zValNames'][0]

In [ ]:
[e[0:1000,:].shape for e in ethogram_2[0:4]]

In [ ]:


fig, axes = plt.subplots(4, 1, figsize=(20,10))
ethogram_subset = [e[0:1000,:] for e in ethogram_2[0:4]]

for e, name, ax in zip(ethogram_subset, wshedfile['zValNames'][0], axes.flatten()):
  print(e.shape)
  ax.imshow(e.T, aspect='auto', cmap=mmpy.gencmap())
  ax.set_title(name[0][0])
  ax.set_yticks([i for i in range(1, wregs.max()+1, 4)])
  ax.set_yticklabels(['Region %i'%(j+1) for j in range(1, wregs.max()+1, 4)])

  # xticklocs = [6000*i for i in range(3)]
  # ax.set_xticks(xticklocs)
  # ax.set_xticklabels([j/(6000) for j in xticklocs])

ax.set_xlabel('Time (min)')
plt.savefig("tmp5.png",dpi=300)

In [ ]:
wshedfile = hdf5storage.loadmat('%s/%s/zVals_wShed_groups.mat'%(parameters.projectPath, parameters.method))

try:
  tqdm._instances.clear()
except:
  pass

# fig, axes = plt.subplots(1, 2, figsize=(10,5))
# zValues = wshedfile['zValues']
# m = np.abs(zValues).max()


# sigma=1.0
# _, xx, density = mmpy.findPointDensity(zValues, sigma, 511, [-m-10, m+10])
# axes[0].imshow(density, cmap=mmpy.gencmap(), extent=(xx[0], xx[-1], xx[0], xx[-1]), origin='lower')
# axes[0].axis('off')
# axes[0].set_title('Method : %s'%parameters.method)
# sc = axes[0].scatter([],[],marker='o', color='k', s=500)


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5,5))
zValues = wshedfile['zValues']
m = np.abs(zValues).max()

In [ ]:
sigma=1.0
_, xx, density = mmpy.findPointDensity(zValues, sigma, 511, [-m-10, m+10])

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5,5))
axes.imshow(density, cmap=mmpy.gencmap(), extent=(xx[0], xx[-1], xx[0], xx[-1]), origin='lower')
axes.axis('off')
axes.set_title('Method : %s'%parameters.method)
sc = axes.scatter([],[],marker='o', color='k', s=500)

def animate(t):
#   t = int(t*clips[h5ind].fps)+tstart
#   axes[1].clear()
#   im = axes[1].imshow(clips[h5ind].get_frame(t/clips[h5ind].fps), cmap='Greys', origin='lower')
#   for conn in connections:
#       axes[1].plot(h5s[h5ind][t, conn, 0], h5s[h5ind][t, conn, 1], 'k-')
#   axes[1].axis('off')
  sc.set_offsets(zValues[int(t)])
  return mplfig_to_npimage(fig) #im, ax

In [ ]:
anim = VideoClip(animate, duration=100) # will throw memory error for more than 100.
plt.close()
anim.ipython_display(fps=100, loop=True, autoplay=True, maxduration=120)

## 8.1 Visualize behavioral map

Run the below code to see the behavioral map in action. 

This may take **2 minutes** to run. 

In [ ]:
wshedfile = hdf5storage.loadmat('%s/%s/zVals_wShed_groups.mat'%(parameters.projectPath, parameters.method))

try:
  tqdm._instances.clear()
except:
  pass

fig, axes = plt.subplots(1, 1, figsize=(5,5))
zValues = wshedfile['zValues']
m = np.abs(zValues).max()


sigma=1.0
_, xx, density = mmpy.findPointDensity(zValues, sigma, 511, [-m-10, m+10])
axes.imshow(density, cmap=mmpy.gencmap(), extent=(xx[0], xx[-1], xx[0], xx[-1]), origin='lower')
axes.axis('off')
axes.set_title('Method : %s'%parameters.method)
sc = axes.scatter([],[],marker='o', color='k', s=500)

h5ind = 0
tstart = 0
# connections = [np.arange(6,10), np.arange(10,14), np.arange(14,18), np.arange(18,22), np.arange(22,26), np.arange(26,30),
#               [2,0,1],[0,3,4,5], [31,3,30]]

def animate(t):
  # t = int(t*clips[h5ind].fps)+tstart
  # axes[1].clear()
  # im = axes[1].imshow(clips[h5ind].get_frame(t/clips[h5ind].fps), cmap='Greys', origin='lower')
  # for conn in connections:
  #     axes[1].plot(h5s[h5ind][t, conn, 0], h5s[h5ind][t, conn, 1], 'k-')
  # axes[1].axis('off')
  sc.set_offsets(zValues[t])
  return mplfig_to_npimage(fig) #im, ax


anim = VideoClip(animate, duration=2) # will throw memory error for more than 100.
plt.close()
anim.ipython_display(fps=15, loop=True, autoplay=True, maxduration=120)


At this point, you know everything you need to know to create a behavioral map for a set of datasets. But this is where we can actually start doing some science! 

Open the project directory on the left pane. We should see some files in *Fly_Leap_mmpy/TSNE* or Fly_Leap_mmpy/UMAP folder. The 2-dimensional embeddings of all the files we've used for this project can be found in the **zVals_wShed_groups.mat** file which can be opened using hdf5storage in Python (as we did in the previous cell) or natively in MATLAB. This is where we can start doing some science!


In [ ]:
plt.figure()
plt.imshow(
wshedfile['density'])

In [ ]:
wshedfile['zValNames']

In [ ]:
wshedfile['zValLens']

In [ ]:
wshedfile['watershedRegions'].shape

In [ ]:

wshedfile['zValues'].shape

In [ ]:
list(wshedfile.keys())

## 8.2 Create region videos



Now we'll try to create region videos - we'll pick contiguous time points that belong in one watershed region, and see what the animals is doing at those times by creating a movie.  

In [ ]:
def makeGroupsAndSegments(watershedRegions, zValLens, min_length=10, max_length=100):

  inds = np.zeros_like(watershedRegions)
  start = 0
  for l in zValLens:
      inds[start:start + l] = np.arange(l)
      start += l
  vinds = np.digitize(np.arange(watershedRegions.shape[0]), bins=np.concatenate([[0], np.cumsum(zValLens)]))

  splitinds = np.where(np.diff(watershedRegions, axis=0) != 0)[0] + 1
  inds = [i for i in np.split(inds, splitinds) if len(i) > min_length and len(i) < max_length]
  wregs = [i[0] for i in np.split(watershedRegions, splitinds) if len(i) > min_length and len(i) < max_length]

  vinds = [i for i in np.split(vinds, splitinds) if len(i) > min_length and len(i) < max_length]
  groups = [np.empty((0, 3), dtype=int)] * watershedRegions.max()

  for wreg, tind, vind in zip(wregs, inds, vinds):
      if np.all(vind == vind[0]):
          groups[wreg - 1] = np.concatenate(
              [groups[wreg - 1], np.array([vind[0], tind[0] + 1, tind[-1] + 1])[None, :]])
  groups = np.array([[g] for g in groups])
  return groups


def makeregionvideo(region, parameters, wshedfile):

  
  animfps=50.0
  subs=2
  submaxframes = 500
  
  
  groups = makeGroupsAndSegments(wshedfile['watershedRegions'][0], wshedfile['zValLens'][0])
  nregs = len(groups)

  region = region-1
  
  outputdir = '%s/%s/region_videos_%i/'%(parameters.projectPath, parameters.method, nregs)
  if not os.path.exists(outputdir):
    os.mkdir(outputdir)
  groups = groups-1
  print('[Region %i] Starting'%(region+1))

  if os.path.isfile(outputdir + 'regions_' + '%.3i' % (region + 1) + '.mp4'):
      print('[Region %i] Already present. '%(region+1))
      return 
    
  tqdm._instances.clear()

  if not groups[region][0].shape[0] or groups[region][0].shape[0] == 1:
      print('[Region %i] No frames in groups.'%(region+1))
      return

  nframes = np.atleast_1d(np.diff(groups[region][0][:, 1:], axis=1).squeeze())
  if np.sum(nframes < submaxframes) == 0:
      print('[Region %i] All frames sequences more than length %i.'%
            (region+1, submaxframes))
      return

  nplots = min(subs * subs, np.sum(nframes < submaxframes))
  longinds = np.where(nframes < submaxframes)[0]
  selectedclips = longinds[np.argsort(nframes[longinds])[::-1]][:nplots]

  vidindslist = groups[region][0][selectedclips, 0]
  framestoplot = np.array([np.arange(groups[region][0][i, 1], groups[region][0][i, 2]) for i in selectedclips])
  maxsize = max([i.shape[0] for i in framestoplot])

  print('[Region %i] Making region video...'%(region+1))


  subx = max(2, int(np.ceil(np.sqrt(nplots))))
  fig, axes = plt.subplots(subx, subx, figsize=(12, 12))
  fig.subplots_adjust(0, 0, 1.0, 1.0, 0.0, 0.0)

  def make_frame(t):
      j_ = int(t * animfps)
      for i in range(subx * subx):

          ax = axes[i // subx, i % subx]
          ax.clear()
          ax.axis('off')
          if i >= nplots:
              continue
          j = j_ % len(framestoplot[i])
          clip = clips[vidindslist[i]]
          ax.imshow(clip.get_frame(framestoplot[i][j]/clip.fps),
                    cmap='Greys_r', origin='lower')
      return mplfig_to_npimage(fig)

  try:
      tqdm._instances.clear()
  except:
      pass
  
  t1 = time.time()
  animation = VideoClip(make_frame, duration=maxsize / animfps)

  animation.write_videofile(outputdir + 'regions_' + '%.3i' % (region + 1) + '.mp4', fps=animfps, audio=False,
                            threads=1)
  
  print('[Region %i] %i seconds, Saved at %s'%(region+1, time.time()-t1,  outputdir + 'regions_' + '%.3i' % (region + 1) + '.mp4'))




In [ ]:
makeregionvideo(10, parameters, wshedfile)

In [ ]:
# This creates region videos for all the region. This can take a while to run so be careful!
wmax = wshedfile['watershedRegions'].max()
print(wmax)
for i in range(1, wmax+1):
    makeregionvideo(i, parameters, wshedfile)

In [ ]:
# Set region below to see your video.
region = 10

print('Region %i'%region)
from IPython.display import HTML
from base64 import b64encode
outputdir = '%s/%s/region_vidoes_%i/'%(parameters.projectPath, parameters.method, wshedfile['watershedRegions'].max())
mp4 = open(outputdir + 'regions_' + '%.3i' % (region) + '.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls loop autoplay>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)




You can also zip your project folder by calling 
```!zip -r Fly_Leap_mmpy.zip Fly_Leap_mmpy``` 
and download the folder on your local computer to play around with it. 

In [ ]:
# !zip -r Fly_Leap_mmpy.zip Fly_Leap_mmpy

# Transition Matrix

In [ ]:
wregs = wshedfile['watershedRegions'][0]
wregs = np.split(wregs, np.cumsum(wshedfile['zValLens'][0])[:-1])
transitions = [mmpy.demoutils.getTransitions(w[w>0]) for w in wregs]


In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16,5))

statevals = np.arange(1,wshedfile['watershedRegions'].max()+2)
for ax, d in zip(axes.flatten(), [1, 10, 100, 1000]):
    F = mmpy.demoutils.makeTransitionMatrix(np.concatenate(transitions), d)
    ax.imshow(F, cmap='PuRd', extent=(statevals[0], statevals[-1], statevals[0], statevals[-1]))
    ax.set_xlabel('Initial State') 
    ax.set_ylabel('Final State')
plt.savefig('transition_matrix.png', dpi=300)

In [ ]:
_ = mmpy.demoutils.plotLaggedEigenvalues(transitions)
plt.savefig('eigenvalues.png', dpi=300)